In [1]:
# mencari directory saat ini
import os
os.getcwd()

'C:\\Users\\Vibrastic\\Documents'

In [2]:
# Merubah directory
os.chdir('D:\Folder Pribadi Asisten\Bagus\Training')
# Merubah directory (beneran)
os.chdir('D:/Folder Pribadi Asisten/Bagus/Training')

In [3]:
from keras import layers
import keras
import keras.backend as K
from matplotlib import pyplot as plt
from pylab import rcParams
from keras.layers import Activation, Dense,Dot,merge, Dropout,Lambda,Permute,Multiply,LSTM, Conv2D, Flatten, MaxPooling2D, LSTM,RepeatVector,Reshape,TimeDistributed,UpSampling1D
# import common as com
import keras.models
from keras import backend as K
from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.models import Model
# from keras.layers.wrappers import TimeDistributed

Using TensorFlow backend.
In C:\Users\Vibrastic\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Vibrastic\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Vibrastic\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Vibrastic\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Vibrastic\Anaconda3\lib\site-packages\

In [4]:
width = 128
height = 64
channels = 1
input_dim = 640
timesteps = 10
n_features = 64
lr = 0.001
batch_size = 1

In [5]:
import keras.backend as K
import numpy as np

numpyA = np.ones([1,64])

def multA(x):
    A = K.variable(np.ones([1,64]))

    return K.dot(x,A)
# def multa(x)
def multA_T(x):
    A = K.variable(np.ones([1,10]))

    return K.dot(x,A)
# weight_2 = Lambda(lambda x:x*0.2)
# weight_gru1 = weight_2(gru1)
# output = Lambda(lambda x: K.sum(x, axis=-1))(vec)
# model.add(Lambda(multA))

In [6]:
#-------------------------------------------#
#   注意力模块
#-------------------------------------------#
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, lstm_units)

    # (batch_size, time_steps, lstm_units) -> (batch_size, lstm_units, time_steps)
    a = Permute((2, 1))(inputs)

    # 对最后一维进行全连接
    # (batch_size, lstm_units, time_steps) -> (batch_size, lstm_units, time_steps)
    a = Dense(10, activation='softmax')(a)

    # (batch_size, lstm_units, time_steps) -> (batch_size, time_steps, lstm_units)
    a_probs = Permute((2, 1), name='attention_vec')(a)

    # 相乘
    # 相当于获得每一个step中，每个维度在所有step中的权重
    output_attention_mul = Multiply()([inputs,a_probs])
#     output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul

# Generatoe Encoder

In [7]:
import tensorflow as tf
input_layer = Input(name='input', shape=(input_dim, ))

input_layer1 = Reshape((timesteps,64))(input_layer)#[10,64]
att = TimeDistributed(Dense(64))(input_layer1)#[B,T,F]
att = Activation('sigmoid')(att)

att1 = Lambda(lambda x: K.sum(x, axis=-1))(att)
att1 = Reshape([-1,1])(att1)
att1 = Lambda(multA)(att1)
att1 = Reshape([-1,64])(att1)

att = Lambda(lambda x:x*64)(att)
att1 = Lambda(lambda x:1/x)(att1)
freq_fnorm = Multiply()([att,att1])

h = Reshape((640,))(freq_fnorm)
# h= Flatten()(cnn_input)
# print(h.shape)
h = Dense(512)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(128)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(128)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

h = Dense(8)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

  
g_e = keras.models.Model(inputs=input_layer, outputs=h)


g_e.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 10, 64)       0           input[0][0]                      
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 10, 64)       4160        reshape_1[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 10, 64)       0           time_distributed_1[0][0]         
____________________________________________________________________________________________

# Generator

In [8]:
input_layer = Input(name='input', shape=(input_dim,))

x = g_e(input_layer)


y = Dense(128)(x)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(128)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(256)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(256)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Dense(input_dim)(y)


g = keras.models.Model(inputs=input_layer, outputs=y)

g.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 640)               0         
_________________________________________________________________
model_1 (Model)              (None, 8)                 585064    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               1152      
_________________________________________________________________
batch_normalization_7 (Batch (None, 128)               512       
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512 

# Encoder

In [9]:
input_layer = Input(name='input', shape=(input_dim, ))

input_layer1 = Reshape((timesteps,64))(input_layer)#[10,64]
att = TimeDistributed(Dense(64))(input_layer1)#[B,T,F]
att = Activation('sigmoid')(att)

att1 = Lambda(lambda x: K.sum(x, axis=-1))(att)
att1 = Reshape([-1,1])(att1)
att1 = Lambda(multA)(att1)
att1 = Reshape([-1,64])(att1)

att = Lambda(lambda x:x*64)(att)
att1 = Lambda(lambda x:1/x)(att1)
freq_fnorm = Multiply()([att,att1])
cnn_input = Multiply()([input_layer1,freq_fnorm])

z = Reshape((640,))(cnn_input)

z = Dense(512)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(256)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(256)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(128)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(128)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

z = Dense(8)(z)
z = BatchNormalization()(z)
z = Activation('relu')(z)

encoder = keras.models.Model(input_layer, z)
encoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 10, 64)       0           input[0][0]                      
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 10, 64)       4160        reshape_5[0][0]                  
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 10, 64)       0           time_distributed_2[0][0]         
____________________________________________________________________________________________

# Feature Extractor

In [10]:
input_layer = Input(name='input', shape=(input_dim,))

f = Dense(512)(input_layer)
f = BatchNormalization()(f)
f = Activation('relu')(f)

h = Dense(256)(h)
h = BatchNormalization()(h)
h = Activation('relu')(h)

f = Dense(256)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

f = Dense(128)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

f = Dense(128)(f)
f = BatchNormalization()(f)
f = Activation('relu')(f)

feature_extractor = keras.models.Model(input_layer, f)

feature_extractor.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 640)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               328192    
_________________________________________________________________
batch_normalization_18 (Batc (None, 512)               2048      
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
batch_normalization_20 (Batc (None, 256)               1024      
_________________________________________________________________
activation_22 (Activation)   (None, 256)               0   

# GAN trainer

In [11]:
class AdvLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AdvLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori_feature = feature_extractor(x[0])
        gan_feature = feature_extractor(x[1])
        return K.mean(K.square(ori_feature - K.mean(gan_feature, axis=0)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class CntLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CntLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.square(ori - gan))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
class EncLoss(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(EncLoss, self).__init__(**kwargs)

    def call(self, x, mask=None):
        ori = x[0]
        gan = x[1]
        return K.mean(K.square(g_e(ori) - encoder(gan)))

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0], 1)
    
# model for training
input_layer = layers.Input(name='input', shape=(input_dim,))
gan = g(input_layer) # g(x)

adv_loss = AdvLoss(name='adv_loss')([input_layer, gan])
cnt_loss = CntLoss(name='cnt_loss')([input_layer, gan])
enc_loss = EncLoss(name='enc_loss')([input_layer, gan])

gan_trainer = keras.models.Model(input_layer, [adv_loss, cnt_loss, enc_loss])

# loss function
def loss(yt, yp):
    return yp

losses = {
    'adv_loss': loss,
    'cnt_loss': loss,
    'enc_loss': loss,
}

lossWeights = {'cnt_loss': 10.0, 'adv_loss': 1.0, 'enc_loss': 1.0}

# compile
gan_trainer.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=losses, loss_weights=lossWeights)

In [12]:
gan_trainer.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 640)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 640)          1166568     input[0][0]                      
__________________________________________________________________________________________________
adv_loss (AdvLoss)              [(None, 640), (None, 0           input[0][0]                      
                                                                 model_2[1][0]                    
__________________________________________________________________________________________________
cnt_loss (CntLoss)              [(None, 640), (None, 0           input[0][0]                

# Discriminator

In [2]:
input_layer = layers.Input(name='input', shape=(input_dim,))

f = feature_extractor(input_layer)

d = Dense(32)(f)
d = BatchNormalization()(d)
d = Activation('elu')(d)

d = layers.Dense(1, activation='sigmoid', name='d_out')(d)    

d = keras.models.Model(input_layer, d)
d.summary()

NameError: name 'layers' is not defined

In [1]:
d.compile(optimizer=keras.optimizers.Adam(lr=lr), loss='binary_crossentropy')

NameError: name 'd' is not defined